## How to Access PUDL Tables

This notebook is going to walk through 3 different ways to access PUDL tables. These will be three different methods to turn a PUDL table into a panda.DataFrame. There are many other access methods, but these are ones that our team uses often. Similar tools exist to convert SQLite tables into other formats. The Catalyst team typically uses the <a href="https://www.sqlalchemy.org/">SQLAlchemy</a> toolset to interact with the SQLite database.

It is important to note that the PUDL databse has been <a href="https://en.wikipedia.org/wiki/Database_normalization">normalized</a> and organized with the <a href="https://tidyr.tidyverse.org/articles/tidy-data.html">tidy data methodology</a>. Because of that, the individual tables in PUDL are more useful and readable when combined.

This notebook assumes you have access to an instance of PUDL database and have an installed pudl python package.

If you have any questions please reach out to: hello@catalyst.coop

In [ ]:
# import the necessary packages
%load_ext autoreload
%autoreload 2

import pandas as pd
import sqlalchemy as sa
import random
import pudl

### Set up PUDL Engine - Conneciton to the Database

Generate a SQLalchemy databse engine. This step in necessary for each of the access methods.

In [ ]:
pudl_settings = pudl.workspace.setup.get_defaults()
pudl_engine = sa.create_engine(pudl_settings["pudl_db"])

In [ ]:
# see all the tables inside of the database
pudl_engine.table_names()

### 1. Select PUDL Database Tables via a SQLAlchemy Select

In [ ]:
# little helper function
def get_full_sql_table(table_name, engine):
    """Get a full table from a SQL database."""
    # generate table metadata
    md = sa.MetaData()
    md.reflect(engine)
    pt = md.tables
    
    # make a sql select statement
    select = sa.sql.select([pt[table_name], ])
    # read the sql select into a dataframe
    df = pd.read_sql(select, engine)
    return df

In [ ]:
# select a random table
table_name = random.choice(pudl_engine.table_names())

In [ ]:
# grab the full table out 
print(f"grabbing {table_name}")
table_df = get_full_sql_table(table_name=table_name, engine=pudl_engine)
table_df.head()

### 2. Select PUDL Database tables with SQL

If you want to <a href="https://www.geeksforgeeks.org/what-are-the-best-ways-to-write-a-sql-query/">write SQL queries</a> directly into the `read_sql()` function, this is the base format for that.

In [ ]:
table_df = pd.read_sql(
    """SELECT * FROM fuel_receipts_costs_eia923;""",
    pudl_engine)

In [ ]:
table_df.head()

### 3. Use PUDL-compiled output tables

If you want to access de-normalized tables, we've built an access methodology that saves access methods for most denormalized tables in PUDL and analysis build ontop of PUDL tables. There is a whole other notebook that covers the output tables so if you want more info on that.

In [ ]:
pudl_out = pudl.output.pudltabl.PudlTabl(pudl_engine)

In [ ]:
bga_eia860 = pudl_out.bga_eia860()
bga_eia860.sample(5)

In [ ]:
frc_eia923 = pudl_out.frc_eia923()
frc_eia923.head()